# Movie title scraper
Scrapes tables for movie titles

In [13]:
import urllib.request
import pandas as pd
import sqlite3

Connect to PostgreSQL

In [14]:
con = sqlite3.connect("database")
cur = con.cursor()

Extract values from movie_lists table

In [15]:
res = cur.execute("SELECT * FROM movie_lists")
movie_lists = res.fetchall()
movie_lists

[(1, 1950, 'Hindi'),
 (2, 1950, 'British'),
 (3, 1950, 'Telugu'),
 (4, 1950, 'Argentine'),
 (5, 1950, 'Italian'),
 (6, 1950, 'South_Korean'),
 (7, 1950, 'Brazilian'),
 (8, 1950, 'Egyptian'),
 (9, 1950, 'Pakistani'),
 (10, 1950, 'Soviet'),
 (11, 1950, 'Japanese'),
 (12, 1950, 'Ollywood'),
 (13, 1950, 'Marathi'),
 (14, 1950, 'Hong_Kong'),
 (15, 1950, 'American'),
 (16, 1950, 'French'),
 (17, 1950, 'Mexican'),
 (19, 1950, 'Israeli'),
 (20, 1950, 'Spanish'),
 (21, 1950, 'Tamil'),
 (22, 1951, 'Hindi'),
 (23, 1951, 'British'),
 (24, 1951, 'Telugu'),
 (25, 1951, 'Argentine'),
 (26, 1951, 'Italian'),
 (27, 1951, 'South_Korean'),
 (28, 1951, 'Brazilian'),
 (29, 1951, 'Egyptian'),
 (30, 1951, 'Pakistani'),
 (31, 1951, 'Soviet'),
 (32, 1951, 'Japanese'),
 (33, 1951, 'Ollywood'),
 (34, 1951, 'Marathi'),
 (35, 1951, 'Hong_Kong'),
 (36, 1951, 'American'),
 (37, 1951, 'French'),
 (38, 1951, 'Mexican'),
 (40, 1951, 'Israeli'),
 (41, 1951, 'Spanish'),
 (42, 1951, 'Tamil'),
 (43, 1952, 'Argentine'),
 (4

In [19]:
log = []
for list_id, year, country in movie_lists:
    url = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_" + country + "_films_of_" + str(year))
    content = url.read().decode()
    try:
        table_list = pd.read_html(content)
    except:
        pass
    temp_titles = []
    for table in table_list:
        df_temp1 = pd.DataFrame(table)
        if "Title" in list(df_temp1.columns):
            temp_titles.extend(df_temp1['Title'].tolist())
    to_db = list(set([(title, year, country, list_id) for title in temp_titles]))
    i = 0
    for entry in to_db:
        cur.execute("INSERT INTO movie_titles (title, year, country, list_id) VALUES (?, ?, ?, ?)", entry)
        i += 1
        if i == 100:
            con.commit()
            i = 0
    con.commit()
    print(f"Retrieved {len(to_db)} {country} movies from {year}.")
    log.append(f"Retrieved {len(to_db)} {country} movies from {year}.")

Retrieved 92 Hindi movies from 1950.
Retrieved 86 British movies from 1950.
Retrieved 18 Telugu movies from 1950.
Retrieved 0 Argentine movies from 1950.
Retrieved 104 Italian movies from 1950.
Retrieved 0 South_Korean movies from 1950.
Retrieved 20 Brazilian movies from 1950.
Retrieved 3 Egyptian movies from 1950.
Retrieved 14 Pakistani movies from 1950.
Retrieved 13 Soviet movies from 1950.
Retrieved 10 Japanese movies from 1950.
Retrieved 3 Ollywood movies from 1950.
Retrieved 0 Marathi movies from 1950.
Retrieved 49 Hong_Kong movies from 1950.
Retrieved 445 American movies from 1950.
Retrieved 74 French movies from 1950.
Retrieved 25 Mexican movies from 1950.
Retrieved 3 Israeli movies from 1950.
Retrieved 13 Spanish movies from 1950.
Retrieved 15 Tamil movies from 1950.
Retrieved 90 Hindi movies from 1951.
Retrieved 84 British movies from 1951.
Retrieved 23 Telugu movies from 1951.
Retrieved 0 Argentine movies from 1951.
Retrieved 128 Italian movies from 1951.
Retrieved 0 South_Ko

[(4, 1950, 'Argentine'),
 (25, 1951, 'Argentine'),
 (43, 1952, 'Argentine'),
 (64, 1953, 'Argentine'),
 (83, 1954, 'Argentine'),
 (103, 1955, 'Argentine'),
 (125, 1956, 'Argentine'),
 (143, 1957, 'Argentine'),
 (164, 1958, 'Argentine'),
 (184, 1959, 'Argentine'),
 (202, 1960, 'Argentine'),
 (210, 1961, 'Argentine'),
 (233, 1962, 'Argentine'),
 (256, 1963, 'Argentine'),
 (279, 1964, 'Argentine'),
 (304, 1965, 'Argentine'),
 (327, 1966, 'Argentine'),
 (351, 1967, 'Argentine'),
 (376, 1968, 'Argentine'),
 (399, 1969, 'Argentine'),
 (423, 1970, 'Argentine'),
 (453, 1971, 'Argentine'),
 (481, 1972, 'Argentine'),
 (509, 1973, 'Argentine'),
 (536, 1974, 'Argentine'),
 (562, 1975, 'Argentine'),
 (588, 1976, 'Argentine'),
 (614, 1977, 'Argentine'),
 (640, 1978, 'Argentine'),
 (665, 1979, 'Argentine'),
 (691, 1980, 'Argentine'),
 (746, 1982, 'Argentine'),
 (772, 1983, 'Argentine'),
 (796, 1984, 'Argentine'),
 (873, 1987, 'Argentine'),
 (898, 1988, 'Argentine'),
 (949, 1990, 'Argentine'),
 (974, 

Fix 1

In [34]:
res = cur.execute('''SELECT id, year, country FROM movie_lists
LEFT JOIN (SELECT list_id, COUNT(*) AS count
           FROM movie_titles
           GROUP BY list_id) rs on movie_lists.id = rs.list_id
WHERE rs.count IS NULL
ORDER BY country''')

fix = res.fetchall()

for list_id, year, country in fix:
    url = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_" + country + "_films_of_" + str(year))
    content = url.read().decode()
    try:
        table_list = pd.read_html(content)
    except:
        pass
    temp_titles = []
    for table in table_list:
        df_temp1 = pd.DataFrame(table)
        for header in [(str(x), x) for x in list(df_temp1.columns)]:
            if "Title" in header[0]:
                temp_titles.extend(df_temp1[header[1]].tolist())
    to_db = list(set([(title, year, country, list_id) for title in temp_titles]))
    i = 0
    for entry in to_db:
        cur.execute("INSERT INTO movie_titles (title, year, country, list_id) VALUES (?, ?, ?, ?)", entry)
        i += 1
        if i == 100:
            con.commit()
            i = 0
    con.commit()
    print(f"Retrieved {len(to_db)} {country} movies from {year}.")
    log.append(f"Retrieved {len(to_db)} {country} movies from {year}.")

Retrieved 67 Argentine movies from 1950.
Retrieved 61 Argentine movies from 1951.
Retrieved 40 Argentine movies from 1952.
Retrieved 42 Argentine movies from 1953.
Retrieved 50 Argentine movies from 1954.
Retrieved 52 Argentine movies from 1955.


KeyboardInterrupt: 

Fix 2

In [36]:
res = cur.execute('''SELECT id, year, country FROM movie_lists
LEFT JOIN (SELECT list_id, COUNT(*) AS count
           FROM movie_titles
           GROUP BY list_id) rs on movie_lists.id = rs.list_id
WHERE rs.count IS NULL
ORDER BY country''')

fix = res.fetchall()

for list_id, year, country in fix:
    url = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_" + country + "_films_of_" + str(year))
    content = url.read().decode()
    try:
        table_list = pd.read_html(content)
    except:
        pass
    temp_titles = []
    for table in table_list:
        df_temp1 = pd.DataFrame(table)
        for header in [(str(x), x) for x in list(df_temp1.columns)]:
            if "Film" in header[0]:
                temp_titles.extend(df_temp1[header[1]].tolist())
    to_db = list(set([(title, year, country, list_id) for title in temp_titles]))
    i = 0
    for entry in to_db:
        cur.execute("INSERT INTO movie_titles (title, year, country, list_id) VALUES (?, ?, ?, ?)", entry)
        i += 1
        if i == 100:
            con.commit()
            i = 0
    con.commit()
    print(f"Retrieved {len(to_db)} {country} movies from {year}.")
    log.append(f"Retrieved {len(to_db)} {country} movies from {year}.")

Retrieved 0 Argentine movies from 2018.
Retrieved 1 Bangladeshi movies from 1956.
Retrieved 24 Bangladeshi movies from 1967.
Retrieved 34 Bangladeshi movies from 1968.
Retrieved 57 Bangladeshi movies from 1969.
Retrieved 43 Bangladeshi movies from 1970.
Retrieved 0 Egyptian movies from 2017.
Retrieved 0 Gujarati movies from 2015.
Retrieved 0 Gujarati movies from 2016.
Retrieved 0 Gujarati movies from 2017.
Retrieved 0 Gujarati movies from 2018.
Retrieved 3 Gujarati movies from 2019.
Retrieved 1 Gujarati movies from 2020.
Retrieved 0 Gujarati movies from 2021.
Retrieved 1 Gujarati movies from 2022.
Retrieved 11 Malayalam movies from 1961.
Retrieved 16 Malayalam movies from 1962.
Retrieved 13 Malayalam movies from 1963.
Retrieved 19 Malayalam movies from 1964.
Retrieved 31 Malayalam movies from 1965.
Retrieved 28 Malayalam movies from 1966.
Retrieved 41 Malayalam movies from 1967.
Retrieved 33 Malayalam movies from 1968.
Retrieved 33 Malayalam movies from 1969.
Retrieved 40 Malayalam mov

Fix 3

In [38]:
res = cur.execute('''SELECT id, year, country FROM movie_lists
LEFT JOIN (SELECT list_id, COUNT(*) AS count
           FROM movie_titles
           GROUP BY list_id) rs on movie_lists.id = rs.list_id
WHERE rs.count IS NULL
ORDER BY country''')

fix = res.fetchall()

for list_id, year, country in fix:
    url = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_" + country + "_films_of_" + str(year))
    content = url.read().decode()
    try:
        table_list = pd.read_html(content)
    except:
        pass
    temp_titles = []
    for table in table_list:
        df_temp1 = pd.DataFrame(table)
        for header in [(str(x), x) for x in list(df_temp1.columns)]:
            if "title" in header[0]:
                temp_titles.extend(df_temp1[header[1]].tolist())
    to_db = list(set([(title, year, country, list_id) for title in temp_titles]))
    i = 0
    for entry in to_db:
        cur.execute("INSERT INTO movie_titles (title, year, country, list_id) VALUES (?, ?, ?, ?)", entry)
        i += 1
        if i == 100:
            con.commit()
            i = 0
    con.commit()
    print(f"Retrieved {len(to_db)} {country} movies from {year}.")
    log.append(f"Retrieved {len(to_db)} {country} movies from {year}.")

Retrieved 0 Argentine movies from 2018.
Retrieved 0 Egyptian movies from 2017.
Retrieved 0 Gujarati movies from 2015.
Retrieved 0 Gujarati movies from 2016.
Retrieved 0 Gujarati movies from 2017.
Retrieved 0 Gujarati movies from 2018.
Retrieved 0 Gujarati movies from 2021.
Retrieved 0 Nigerian movies from 2001.
Retrieved 0 Nigerian movies from 2002.
Retrieved 0 Nigerian movies from 2003.
Retrieved 0 Nigerian movies from 2008.
Retrieved 0 Nigerian movies from 2009.
Retrieved 7 South_Korean movies from 1950.
Retrieved 15 South_Korean movies from 1951.
Retrieved 13 South_Korean movies from 1952.
Retrieved 15 South_Korean movies from 1953.
Retrieved 33 South_Korean movies from 1954.
Retrieved 31 South_Korean movies from 1955.
Retrieved 58 South_Korean movies from 1956.
Retrieved 75 South_Korean movies from 1957.
Retrieved 38 South_Korean movies from 1992.
Retrieved 0 Turkish movies from 1970.
Retrieved 0 Turkish movies from 1971.
Retrieved 0 Turkish movies from 1972.
Retrieved 0 Turkish mo

Fix 4

In [39]:
res = cur.execute('''SELECT id, year, country FROM movie_lists
LEFT JOIN (SELECT list_id, COUNT(*) AS count
           FROM movie_titles
           GROUP BY list_id) rs on movie_lists.id = rs.list_id
WHERE rs.count IS NULL
ORDER BY country''')

fix = res.fetchall()

for list_id, year, country in fix:
    url = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_" + country + "_films_of_" + str(year))
    content = url.read().decode()
    try:
        table_list = pd.read_html(content)
    except:
        pass
    temp_titles = []
    for table in table_list:
        df_temp1 = pd.DataFrame(table)
        for header in [(str(x), x) for x in list(df_temp1.columns)]:
            if "Name" in header[0]:
                temp_titles.extend(df_temp1[header[1]].tolist())
    to_db = list(set([(title, year, country, list_id) for title in temp_titles]))
    i = 0
    for entry in to_db:
        cur.execute("INSERT INTO movie_titles (title, year, country, list_id) VALUES (?, ?, ?, ?)", entry)
        i += 1
        if i == 100:
            con.commit()
            i = 0
    con.commit()
    print(f"Retrieved {len(to_db)} {country} movies from {year}.")
    log.append(f"Retrieved {len(to_db)} {country} movies from {year}.")

Retrieved 0 Argentine movies from 2018.
Retrieved 0 Egyptian movies from 2017.
Retrieved 14 Gujarati movies from 2015.
Retrieved 45 Gujarati movies from 2016.
Retrieved 61 Gujarati movies from 2017.
Retrieved 59 Gujarati movies from 2018.
Retrieved 24 Gujarati movies from 2021.
Retrieved 0 Nigerian movies from 2001.
Retrieved 0 Nigerian movies from 2002.
Retrieved 0 Nigerian movies from 2003.
Retrieved 0 Nigerian movies from 2008.
Retrieved 0 Nigerian movies from 2009.
Retrieved 224 Turkish movies from 1970.
Retrieved 265 Turkish movies from 1971.
Retrieved 301 Turkish movies from 1972.
Retrieved 208 Turkish movies from 1973.
Retrieved 188 Turkish movies from 1974.
Retrieved 225 Turkish movies from 1975.
Retrieved 164 Turkish movies from 1976.
Retrieved 126 Turkish movies from 1977.
Retrieved 126 Turkish movies from 1978.
Retrieved 205 Turkish movies from 1979.
